# Imports

In [1]:
from typing import Optional
import os
from time import time

In [2]:
import pandas as pd
import tensorflow as tf
from keras.api.models import load_model
# from keras.api.models import Model as KerasModel

In [3]:
from experiments.utils import (
    generate_layers_to_relax,
    linear_constraints_to_dict,
    store_data,
)

from utils.milp import codify_network, relax_model
from utils.explanations import get_explanation_range, get_minimal_explanation


# Utils

In [ ]:

def benchmark(
    results_path: str, 
    layers_idx: Optional[list[int]] = None
):
    # Datasets e modelos
    datasets = [
        # {"nome": "iris", "n_classes": 3},
        # {"nome": "wine", "n_classes": 3},
        # {"nome": "breast_cancer", "n_classes": 2},
        # {"nome": "glass", "n_classes": 5},
        {"nome": "digits", "n_classes": 10},
        # {"nome":"mnist", "n_classes":10}, # rodar individualmente
    ]
    modelos = [
        # "model_1layers_20neurons.h5",
        # "model_2layers_20neurons.h5",
        "model_3layers_20neurons.h5",
    ]

    # definir array de deltas a seres utilizados
    # deltas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

    # para cada dataset
    for dataset in datasets:
        data = pd.read_csv(f"datasets/{dataset['nome']}/data.csv")
        # para cada modelo do dataset
        for modelo in modelos:

            # carregar modelo em formato h5
            keras_model = load_model(
                f"datasets/{dataset['nome']}/models/{modelo}", compile=False
            )

            # 1 - gerar modelos MILP
            method = "fischetti"
            model_milp, output_bounds = codify_network(keras_model, data, method, True)

            # 2 - gerar modelo MILP relaxado
            # model_milp_relaxed = relax_model(
            #     model=model_milp,
            #     neural_network=keras_model, # type: ignore
            #     layers_to_relax=generate_layers_to_relax(
            #         neural_network=keras_model, # type: ignore
            #         layers_idx=layers_idx, # type: ignore
            # ))

            for index in range(len(data)):
                # if index < 64:  # type: ignore
                #     continue
                
                if index >= 100:  # type: ignore
                    break
                print(f"Dataset: {dataset['nome']} Modelo: {modelo} Instancia: {index}")

                # Predicao
                network_input = tf.reshape(tf.constant(data.iloc[index, :-1]), (1, -1))  # type: ignore
                network_output = tf.argmax(keras_model.predict(tf.constant(network_input), verbose=0)[0])  # type: ignore

                # Explicacao
                init = time()
                (explanation, _) = get_minimal_explanation(
                    model_milp.clone(),
                    network_input,
                    network_output,
                    n_classes=dataset["n_classes"],
                    method=method,
                    output_bounds=output_bounds,
                )
                end = time()

                original = {
                    "time_milp": end - init,
                    "len_milp": len(explanation),
                    "explanation": linear_constraints_to_dict(explanation),
                }

                store_data(
                    f"{results_path}/{dataset['nome']}.csv",
                    [
                        {
                            "dataset": dataset["nome"],
                            "modelo": modelo,
                            "instance": index,
                            **{f"original_{k}": v for k, v in original.items()},
                            # **{f"relaxed_{k}": v for k, v in relaxed.items()},
                        }
                    ],
                )

# Experiments

In [5]:
path = os.path.join(
    "results",
    "original",
    "get_minimal_explanation",
)

os.makedirs(path, exist_ok=True)

benchmark(path)

Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 64
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 65
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 66
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 67
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 68
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 69
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 70
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 71
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 72
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 73
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 74
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 75
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 76
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 77
Dataset: digits Modelo: model_3layers_20neurons.h5 Instancia: 78
Dataset: digits Modelo: m